In [4]:
import pandas as pd
import os
import math

In [5]:
fte = pd.read_csv(os.path.expanduser('~/Downloads/spi_matches.csv'))

In [ ]:
fte.columns

## Put two things online for every league
1. Table
2. Transformed aggregate version w/ one team like cte on sql query

In [ ]:
def mvp(df, date, home_team, away_team, home_goals, away_goals, home_xg, away_xg, home_nsxg, away_nxsg, ):
    #home
    home = df.groupby(home_team).apply(
        lambda x: (x[home_goals] > x[away_goals]).sum() * 3 + (x[home_goals] == x[away_goals]).sum()).reset_index(name='points')
    pt = pd.pivot_table(df, values = [home_goals, away_goals], index = home_team, aggfunc=sum)
    home = home.merge(pt.reset_index(), on = home_team)
    pt = pd.pivot_table(df, values = date, index = home_team, aggfunc=pd.Series.nunique)
    home = home.merge(pt.reset_index(), on = home_team)
    home = home.rename(columns = {home_team:'team', away_goals:'conceded', home_goals: 'scored', date: 'played'})
    if results == 'home':
        t = home
#away
    away = df.groupby(away_team).apply(
        lambda x: (x[away_goals] > x[home_goals]).sum() * 3 + (x[home_goals] == x[away_goals]).sum()).reset_index(name='points')
    pt = pd.pivot_table(df, values = [home_goals, away_goals], index = away_team, aggfunc=sum)
    away = away.merge(pt.reset_index(), on = away_team)
    pt = pd.pivot_table(df, values = date, index = away_team, aggfunc=pd.Series.nunique)
    away = away.merge(pt.reset_index(), on = away_team)
    away = away.rename(columns = {away_team:'team', away_goals:'scored', home_goals: 'conceded', date: 'played'})
    if results == 'away':
        t = away
#merge
    t = pd.concat([home, away], ignore_index=True)
    t = t.groupby('team').sum()
    t['goal_difference'] = t['scored'] - t['conceded']
    return t

In [6]:
def fte_last_season(path, league_dict):
    df = pd.read_csv(path)
    for key in league_dict:
        q = ''.join(['league_id == ', league_dict[key]])
        league = df.query(q).reset_index()
        num_teams = len(league.loc[:50, 'team1'].unique())
        i = math.floor(league.shape[0]/(num_teams * (num_teams - 1))) - 1
        return league.loc[((num_teams * (num_teams - 1)) * i):,:]

In [7]:
path = os.path.expanduser('~/Downloads/spi_matches.csv')
# league_dict = {'england':'2411'}
league_dict = {'spain':'1869'}
#, 'italy':'1854','germany':'1845', 'france':'1843'}
df = fte_last_season(path, league_dict)

In [22]:
df

,index,date,league_id,league,team1,team2,spi1,spi2,prob1,prob2,...,importance1,importance2,score1,score2,xg1,xg2,nsxg1,nsxg2,adj_score1,adj_score2
1140,23400,2019-08-16,1869,Spanish Primera Division,Athletic Bilbao,Barcelona,73.34,91.18,0.1873,0.5939,...,35.6,71.4,1.0,0.0,0.53,1.31,0.64,1.60,1.05,0.00
1141,23468,2019-08-17,1869,Spanish Primera Division,Celta Vigo,Real Madrid,67.73,85.18,0.2333,0.5457,...,24.9,55.7,1.0,3.0,1.63,1.79,1.38,2.33,0.84,2.90
1142,23485,2019-08-17,1869,Spanish Primera Division,Valencia,Real Sociedad,77.77,73.51,0.5380,0.2171,...,52.3,38.1,1.0,1.0,2.11,1.47,2.09,1.63,1.05,0.84
1143,23486,2019-08-17,1869,Spanish Primera Division,Mallorca,Eibar,63.73,73.21,0.3664,0.3552,...,38.3,31.5,2.0,1.0,1.56,0.97,1.17,1.66,2.10,1.05
1144,23500,2019-08-17,1869,Spanish Primera Division,Villarreal,Granada,71.24,63.83,0.5492,0.2062,...,27.4,35.4,4.0,4.0,2.09,1.85,0.98,1.12,4.14,4.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1515,33076,2020-05-24,1869,Spanish Primera Division,Levante,Getafe,61.68,81.31,0.2847,0.4281,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1516,33077,2020-05-24,1869,Spanish Primera Division,Granada,Athletic Bilbao,70.41,71.79,0.4355,0.2537,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1517,33078,2020-05-24,1869,Spanish Primera Division,Osasuna,Mallorca,70.37,60.47,0.5685,0.1910,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1518,33083,2020-05-24,1869,Spanish Primera Division,Alavés,Barcelona,66.83,90.33,0.1688,0.6246,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
fte.columns

Index(['index', 'date', 'league_id', 'league', 'team1', 'team2', 'spi1',
       'spi2', 'prob1', 'prob2', 'probtie', 'proj_score1', 'proj_score2',
       'importance1', 'importance2', 'score1', 'score2', 'xg1', 'xg2', 'nsxg1',
       'nsxg2', 'adj_score1', 'adj_score2'],
      dtype='object')

In [8]:
def fte_mvp(df):
    home = df.groupby('team1').apply(
        lambda x: (x['score1'] > x['score2']).sum() * 3 + (x['score1'] == x['score2']).sum()).reset_index(name='points')
    pt = pd.pivot_table(df, values = ['score1', 'score2', 'xg1', 'xg2', 'nsxg1', 'nsxg2', 'adj_score1','adj_score2'], 
                        index = 'team1', aggfunc=sum)
    home = home.merge(pt.reset_index(), on = 'team1')
    pt = pd.pivot_table(df, values = 'date', index = 'team1', aggfunc=pd.Series.nunique)
    home = home.merge(pt.reset_index(), on = 'team1')
    cols = {'date':'played', 'team1':'team', 'score1':'scored', 'score2':'conceded', 'xg1':'xg_scored',
            'xg2':'xg_conceded', 'nsxg1':'nsxg_scored', 'nsxg2':'nsxg_conceded', 'adj_score1':'adj_goals_scored',
            'adj_score2':'adj_goals_conceded'}
    home = home.rename(columns = cols)
    away = df.groupby('team2').apply(
        lambda x: (x['score2'] > x['score1']).sum() * 3 + (x['score1'] == x['score2']).sum()).reset_index(name='points')
    pt = pd.pivot_table(df, values = ['score1', 'score2', 'xg1', 'xg2', 'nsxg1', 'nsxg2', 'adj_score1','adj_score2'], 
                        index = 'team2', aggfunc=sum)
    away = away.merge(pt.reset_index(), on = 'team2')
    pt = pd.pivot_table(df, values = 'date', index = 'team2', aggfunc=pd.Series.nunique)
    away = away.merge(pt.reset_index(), on = 'team2')
    cols = {'date':'played', 'team2':'team', 'score2':'scored', 'score1':'conceded', 'xg2':'xg_scored',
            'xg1':'xg_conceded', 'nsxg2':'nsxg_scored', 'nsxg1':'nsxg_conceded', 'adj_score2':'adj_goals_scored',
            'adj_score1':'adj_goals_conceded'}
    away = away.rename(columns = cols)
    t = pd.concat([home, away], ignore_index=True)
    t = t.groupby('team').sum()
    t['goal_difference'] = t['scored'] - t['conceded']
    t['xg_difference'] = t['xg_scored'] - t['xg_conceded']
    t['nsxg_difference'] = t['nsxg_scored'] - t['nsxg_conceded']
    t['adj_goal_difference'] = t['adj_goals_scored'] - t['adj_goals_conceded']
    return t

In [13]:
fte_mvp(df)

,points,adj_goals_scored,adj_goals_conceded,nsxg_scored,nsxg_conceded,scored,conceded,xg_scored,xg_conceded,played,goal_difference,xg_difference,nsxg_difference,adj_goal_difference
team,,,,,,,,,,,,,,
Alavés,32,29.11,37.15,25.25,32.17,29.0,37.0,31.39,41.21,38,-8.0,-9.82,-6.92,-8.04
Athletic Bilbao,37,28.88,24.15,25.57,31.21,29.0,23.0,30.68,34.35,38,6.0,-3.67,-5.64,4.73
Atletico Madrid,45,31.48,21.54,36.07,25.94,31.0,21.0,44.48,24.81,38,10.0,19.67,10.13,9.94
Barcelona,58,63.21,31.65,63.70,25.06,63.0,31.0,57.20,29.15,38,32.0,28.05,38.64,31.56
Celta Vigo,26,22.37,33.63,29.61,37.18,22.0,34.0,30.56,33.57,38,-12.0,-3.01,-7.57,-11.26
Eibar,27,27.05,41.03,29.91,29.87,27.0,41.0,28.06,40.39,38,-14.0,-12.33,0.04,-13.98
Espanyol,20,23.73,44.04,28.48,30.24,23.0,46.0,29.50,39.75,38,-23.0,-10.25,-1.76,-20.31
Getafe,46,36.18,25.41,29.68,20.76,37.0,25.0,32.94,28.58,38,12.0,4.36,8.92,10.77
Granada,38,33.80,31.08,22.46,29.39,33.0,32.0,30.83,33.86,38,1.0,-3.03,-6.93,2.72


In [8]:
def fte_england_tb(df):
    t = fte_mvp(df)
    t = t.sort_values(by=['points', 'goal_difference', 'scored'], ascending = False)
    t['position'] = t.reset_index().index + 1
    return t

In [9]:
fte_england_tb(df)

,adj_goals_conceded,adj_goals_scored,conceded,nsxg_conceded,nsxg_scored,played,points,scored,xg_conceded,xg_scored,goal_difference,xg_difference,nsxg_difference,adj_goal_difference,position
team,,,,,,,,,,,,,,,
Liverpool,21.79,66.60,21.0,26.44,61.85,38,82,66.0,29.78,68.72,45.0,38.94,35.41,44.81,1
Manchester City,30.45,68.39,31.0,21.22,79.44,38,57,68.0,28.66,76.87,37.0,48.21,58.22,37.94,2
Leicester City,29.08,53.87,28.0,36.21,44.03,38,53,58.0,39.10,49.48,30.0,10.38,7.82,24.79,3
Chelsea,40.26,51.77,39.0,26.97,50.95,38,48,51.0,31.47,57.62,12.0,26.15,23.98,11.51,4
Manchester United,30.64,44.59,30.0,33.31,41.68,38,45,44.0,32.97,47.66,14.0,14.69,8.37,13.95,5
Wolverhampton,35.17,41.13,34.0,37.64,34.92,38,43,41.0,32.69,44.64,7.0,11.95,-2.72,5.96,6
Sheffield United,25.95,31.22,25.0,36.60,30.92,38,43,30.0,39.88,37.33,5.0,-2.55,-5.68,5.27,7
Tottenham Hotspur,41.58,48.32,40.0,46.10,36.02,38,41,47.0,46.26,45.70,7.0,-0.56,-10.08,6.74,8
Arsenal,37.04,40.97,36.0,41.37,41.27,38,40,40.0,43.29,39.50,4.0,-3.79,-0.10,3.93,9


In [24]:
def fte_spain_tb(df):
    t = fte_mvp(df)

    if t.duplicated(subset='points', keep = False).sum() > 0:
        t['tb'] = 0
        ties = t[t.duplicated(subset='points', keep = False)]['points'].unique()
        for tie in ties:
            teams = t[t['points'] == tie].index.values
            temp = df.loc[df['team1'].isin(teams) & df['team2'].isin(teams)].dropna()
            if len(temp.index) < (len(teams) * (len(teams) - 1)):
                for team in teams: t.at[team, 'tb'] = 1
            else:
                mini = fte_mvp(temp)
                if mini.duplicated(subset = ['points', 'goal_difference'], keep = False).sum() > 0:
                    mini = pd.merge(mini, t[['goal_difference']], how = 'inner', left_index=True, right_index=True, suffixes=['', '_total'])
                    mini = mini.sort_values(by=['points', 'goal_difference', 'goal_difference_total'], ascending = False)
                else:
                    mini = mini.sort_values(by=['points', 'goal_difference'], ascending = False)

                mini['tb'] = 2 * mini.shape[0] - mini.reset_index().index
                t.update(mini['tb'])
        t = t.sort_values(by=['points', 'tb', 'goal_difference', 'scored'], ascending = False)
        t.drop('tb', axis = 1, inplace=True)
    else:
        t = t.sort_values(by='points', ascending = False)
    t['position'] = t.reset_index().index + 1
    return t

In [23]:
t = fte_mvp(df)
teams = t[t['points'] == 33].index.values
df.loc[df['team1'].isin(teams) & df['team2'].isin(teams)]

,index,date,league_id,league,team1,team2,spi1,spi2,prob1,prob2,...,importance1,importance2,score1,score2,xg1,xg2,nsxg1,nsxg2,adj_score1,adj_score2
1191,25018,2019-09-24,1869,Spanish Primera Division,Real Betis,Levante,69.29,65.72,0.4991,0.2585,...,29.1,28.3,3.0,1.0,2.77,2.02,2.25,1.14,3.15,1.05
1456,32050,2020-04-22,1869,Spanish Primera Division,Levante,Real Betis,61.68,73.31,0.3484,0.4002,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
fte_spain_tb(df)

,points,adj_goals_scored,adj_goals_conceded,nsxg_scored,nsxg_conceded,scored,conceded,xg_scored,xg_conceded,played,goal_difference,xg_difference,nsxg_difference,adj_goal_difference,position
team,,,,,,,,,,,,,,,
Barcelona,58,63.21,31.65,63.70,25.06,63.0,31.0,57.20,29.15,38,32.0,28.05,38.64,31.56,1
Real Madrid,56,47.85,19.74,58.70,24.86,49.0,19.0,53.74,26.73,38,30.0,27.01,33.84,28.11,2
Sevilla FC,47,39.54,30.26,39.48,24.26,39.0,29.0,41.35,29.03,38,10.0,12.32,15.22,9.28,3
Real Sociedad,46,45.68,33.36,37.07,35.55,45.0,33.0,43.81,33.65,38,12.0,10.16,1.52,12.32,4
Getafe,46,36.18,25.41,29.68,20.76,37.0,25.0,32.94,28.58,38,12.0,4.36,8.92,10.77,5
Atletico Madrid,45,31.48,21.54,36.07,25.94,31.0,21.0,44.48,24.81,38,10.0,19.67,10.13,9.94,6
Valencia,42,37.84,38.63,30.53,43.27,38.0,39.0,36.11,44.69,38,-1.0,-8.58,-12.74,-0.79,7
Villarreal,38,42.84,39.29,33.97,36.54,44.0,38.0,50.52,38.36,38,6.0,12.16,-2.57,3.55,8
Granada,38,33.80,31.08,22.46,29.39,33.0,32.0,30.83,33.86,38,1.0,-3.03,-6.93,2.72,9
